In [13]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import BeliefPropagation

import bp
import numpy as np
np.set_printoptions(suppress=True)

In [14]:
U = bp.variableNode("Unblocked FEV1", 2, 6, 0.1)
B = bp.variableNode("Small airway clearance", 0.1, 1, 0.1)
FEV1 = bp.variableNode("FEV1", 0.2, 6, 0.1)

In [15]:
graph = BayesianNetwork([(U.name, FEV1.name), (B.name, FEV1.name)])

cpt_fev1 = TabularCPD(
    variable=FEV1.name,
    variable_card=len(FEV1.bins)-1,
    values=np.reshape(bp.calc_cpt(U, B, FEV1),(58,360), order="F"),
    evidence=[B.name, U.name],
    evidence_card=[len(B.bins)-1, len(U.bins)-1],
)

prior_b = TabularCPD(
    variable=B.name,
    variable_card=len(B.bins)-1,
    values=B.marginal(B),
    evidence=[],
    evidence_card=[],
)

prior_u = TabularCPD(
    variable=U.name,
    variable_card=len(U.bins)-1,
    values=U.marginal(U),
    evidence=[],
    evidence_card=[],
)

graph.add_cpds(cpt_fev1, prior_b, prior_u)

graph.check_model()


calculating cpt of shape 58 x 40 x 9 (C x A x B) 


True

In [17]:
bp = BeliefPropagation(graph)
res = bp.query(variables=[U.name], evidence={FEV1.name: 0})
print(f"shape of the response {res.values.shape}")
res.values

  0%|          | 0/1 [00:00<?, ?it/s]

shape of the response (40,)


array([0.21428602, 0.18281378, 0.15414052, 0.12790869, 0.10381917,
       0.08161978, 0.06109644, 0.04206622, 0.02437187, 0.0078775 ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ])

In [19]:
bp = BeliefPropagation(graph)
res = bp.query(variables=[B.name], evidence={FEV1.name: 0})
print(f"shape of the response {res.values.shape}")
res.values

  0%|          | 0/1 [00:00<?, ?it/s]

shape of the response (9,)


array([1., 0., 0., 0., 0., 0., 0., 0., 0.])